In [1]:
!pip install torch torchvision torchaudio

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pandas as pd
import numpy as np

# Завантаження датасету
df = pd.read_csv("Life Expectancy Data.csv")

# Видалення рядків із пропусками
from sklearn.impute import SimpleImputer
imputer=SimpleImputer(missing_values=np.nan,strategy='mean',fill_value=None)
df_nan = df.isnull().sum()
list_col_to_fill = list(df_nan[df_nan > 0].index)
for cl_n in list_col_to_fill:
    df[cl_n] = imputer.fit_transform(df[[cl_n]])

# Вибір ознак (X) і цільової змінної (y)
# Наприклад, прогнозуємо тривалість життя
X = df.drop(columns=["Life expectancy ", "Country"])
y = df["Life expectancy "]

# Масштабування числових ознак
from sklearn.preprocessing import StandardScaler

# Вибір числових колонок
numeric_cols = X.select_dtypes(include=['float64', 'int64']).columns

# Нормалізація
scaler = StandardScaler()
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])
# Перевірка типів даних
print("Типи даних:\n", X.dtypes)

# Визначення категоріальних колонок
categorical_columns = X.select_dtypes(include=['object']).columns
print("Категоріальні колонки:\n", categorical_columns)

# Кодування категоріальних колонок
X = pd.get_dummies(X, columns=categorical_columns, drop_first=True)
X['Status_Developing'] = X['Status_Developing'].astype(int)
print("Розмір після кодування:", X.shape)



# Розділення на тренувальні та тестові дані
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Перетворення в тензори PyTorch
X_train = torch.tensor(X_train.values, dtype=torch.float32)  # Додаємо .values
X_test = torch.tensor(X_test.values, dtype=torch.float32)    # Додаємо .values
y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Архітектура повнозв'язаної нейронної мережі
class FeedForwardNN(nn.Module):
    def __init__(self, input_dim):
        super(FeedForwardNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Ініціалізація моделі
input_dim = X_train.shape[1]
model = FeedForwardNN(input_dim)

# Вибір оптимізатора та функції втрат
criterion = nn.MSELoss()  # Оскільки це регресійна задача
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Навчання моделі
num_epochs = 1000
for epoch in range(num_epochs):
    # Прогноз
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    # Оновлення ваг
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

# Оцінка моделі
with torch.no_grad():
    y_pred_test = model(X_test)
    test_loss = criterion(y_pred_test, y_test)
    print(f'Test Loss: {test_loss.item():.4f}')

from sklearn.metrics import r2_score

# Оцінка моделі
with torch.no_grad():
    y_pred_test = model(X_test).numpy()  # Передбачення для тестових даних
    y_test_numpy = y_test.numpy()  # Перетворення тестових значень у numpy

    # Розрахунок R^2
    r2 = r2_score(y_test_numpy, y_pred_test)
    print(f"Test R^2 Score: {r2:.4f}")

Типи даних:
 Year                               float64
Status                              object
Adult Mortality                    float64
infant deaths                      float64
Alcohol                            float64
percentage expenditure             float64
Hepatitis B                        float64
Measles                            float64
 BMI                               float64
under-five deaths                  float64
Polio                              float64
Total expenditure                  float64
Diphtheria                         float64
 HIV/AIDS                          float64
GDP                                float64
Population                         float64
 thinness  1-19 years              float64
 thinness 5-9 years                float64
Income composition of resources    float64
Schooling                          float64
dtype: object
Категоріальні колонки:
 Index(['Status'], dtype='object')
Розмір після кодування: (2938, 20)
Epoch 10/1000, Loss:

In [3]:
## 2 

In [4]:
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os

# Шлях до папки з даними
data_path = "tom_and_jerry_dataset"

# Перевірка структури датасету
print("Класи в датасеті:", os.listdir(data_path))

# Підготовка трансформацій
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Зміна розміру зображень
    transforms.ToTensor(),  # Перетворення в тензор
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Нормалізація
])

# Завантаження даних
dataset = ImageFolder(root=data_path, transform=transform)

# Розділення даних на тренувальні та тестові
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Завантаження в DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Перевірка класів
print(f"Кількість класів: {len(dataset.classes)}, Класи: {dataset.classes}")


Класи в датасеті: ['jerry', 'tom']
Кількість класів: 2, Класи: ['jerry', 'tom']


In [5]:
# Архітектура CNN
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Ініціалізація моделі
num_classes = len(dataset.classes)
model = CNN(num_classes)

# Навчання моделі
# Використовуйте попередній код навчання


In [6]:
import torch.optim as optim
from sklearn.metrics import accuracy_score

# Функція втрат і оптимізатор
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Навчання моделі
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

# Оцінка моделі
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.numpy())
        y_pred.extend(predicted.numpy())

accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy (from scratch): {accuracy:.4f}')


Epoch 1/5, Loss: 0.2803
Epoch 2/5, Loss: 0.2819
Epoch 3/5, Loss: 0.2627
Epoch 4/5, Loss: 0.1255
Epoch 5/5, Loss: 0.0208
Accuracy (from scratch): 0.9416


In [7]:
from torchvision.models import resnet18

# Завантаження попередньо натренованої моделі
model = resnet18(pretrained=True)

# Оновлення вихідного шару
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Freeze попередні шари
for param in model.parameters():
    param.requires_grad = False
model.fc.weight.requires_grad = True
model.fc.bias.requires_grad = True

# Навчання моделі
# Використовуйте попередній код навчання


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
# Функція втрат і оптимізатор
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Навчання моделі
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

# Оцінка моделі
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.numpy())
        y_pred.extend(predicted.numpy())

accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy (transfer learning): {accuracy:.4f}')


Epoch 1/5, Loss: 0.3962
Epoch 2/5, Loss: 0.5417
Epoch 3/5, Loss: 0.1416
Epoch 4/5, Loss: 0.2747
Epoch 5/5, Loss: 0.2024
Accuracy (transfer learning): 0.8675


In [9]:
## 3

In [10]:
import os
import pandas as pd

# Вкажіть шлях до папки з датасетом
dataset_path = r"review_polarity\txt_sentoken"

# Завантаження позитивних і негативних відгуків
pos_reviews = []
neg_reviews = []

# Читання файлів із позитивними відгуками
for filename in os.listdir(os.path.join(dataset_path, "pos")):
    with open(os.path.join(dataset_path, "pos", filename), "r", encoding="utf-8") as file:
        pos_reviews.append(file.read())

# Читання файлів із негативними відгуками
for filename in os.listdir(os.path.join(dataset_path, "neg")):
    with open(os.path.join(dataset_path, "neg", filename), "r", encoding="utf-8") as file:
        neg_reviews.append(file.read())

# Формування датасету
reviews = pos_reviews + neg_reviews
labels = ["positive"] * len(pos_reviews) + ["negative"] * len(neg_reviews)

df = pd.DataFrame({"review": reviews, "sentiment": labels})
print(df.head())


                                              review sentiment
0  films adapted from comic books have had plenty...  positive
1  every now and then a movie comes along from a ...  positive
2  you've got mail works alot better than it dese...  positive
3   " jaws " is a rare film that grabs your atten...  positive
4  moviemaking is a lot like being the general ma...  positive


In [11]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt_tab')

nltk.download('punkt')
nltk.download('stopwords')

# Стоп-слова
stop_words = set(stopwords.words('english'))

# Функція для очищення тексту
def preprocess_text(text):
    text = text.lower()  # До нижнього регістру
    text = re.sub(r'\W', ' ', text)  # Видалення пунктуації
    text = re.sub(r'\s+', ' ', text)  # Видалення зайвих пробілів
    tokens = word_tokenize(text)  # Токенізація
    filtered_tokens = [word for word in tokens if word not in stop_words]  # Видалення стоп-слів
    return ' '.join(filtered_tokens)

# Передобробка текстів
df["cleaned_review"] = df["review"].apply(preprocess_text)
print(df.head())


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                              review sentiment  \
0  films adapted from comic books have had plenty...  positive   
1  every now and then a movie comes along from a ...  positive   
2  you've got mail works alot better than it dese...  positive   
3   " jaws " is a rare film that grabs your atten...  positive   
4  moviemaking is a lot like being the general ma...  positive   

                                      cleaned_review  
0  films adapted comic books plenty success wheth...  
1  every movie comes along suspect studio every i...  
2  got mail works alot better deserves order make...  
3  jaws rare film grabs attention shows single im...  
4  moviemaking lot like general manager nfl team ...  


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Векторизація текстів
vectorizer = TfidfVectorizer(max_features=5000)  # Обмеження кількості ознак
X = vectorizer.fit_transform(df["cleaned_review"])
y = df["sentiment"]


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix

# Розподіл на тренувальну і тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Преобразування міток класів
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

print("Кількість класів:", label_encoder.classes_)




Кількість класів: ['negative' 'positive']


In [14]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
import torch

# Побудова словника
tokenizer = get_tokenizer(language='ukr')
def yield_tokens(data_iter):
    for tokens in data_iter:
        yield tokens

vocab = build_vocab_from_iterator(yield_tokens(X_train), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

# Перетворення текстів на індекси
def text_pipeline(tokens):
    return vocab(tokens)

# Приведення текстів до одного розміру
def collate_batch(batch):
    texts, labels = zip(*batch)
    texts = [torch.tensor(text_pipeline(tokens), dtype=torch.long) for tokens in texts]
    texts = pad_sequence(texts, batch_first=True, padding_value=0)
    labels = torch.tensor(labels, dtype=torch.long)
    return texts, labels

# Підготовка датасету
train_data = list(zip(X_train, y_train))
test_data = list(zip(X_test, y_test))



C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torchtext\__init__.py:7: SyntaxWarning: invalid escape sequence '\ '
  "\n/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ \n"


OSError: [WinError 127] The specified procedure could not be found

In [ ]:
!pip install torchtext